In [ ]:
import os
import nibabel as nib
from nibabel.affines import apply_affine
import numpy.linalg as npl
import matplotlib.pyplot as plt
plt.style.use('seaborn')

import numpy as np
np.set_printoptions(precision=3, suppress=True)

#### Transformations Between Spaces

In [ ]:
t2s_img = nib.load("Data/SCSeg/20/SC/t2sMerge.nii.gz")
t2s_img_data = t2s_img.get_fdata()
print(t2s_img_data.shape)

t2_img = nib.load("Data/SCSeg/20/SC/t2.nii.gz")
t2_img_data = t2_img.get_fdata()
print(t2_img_data.shape)

In [ ]:
def show_slices(slices):
   """ Function to display row of image slices """
   fig, axes = plt.subplots(1, len(slices))
   for i, slice in enumerate(slices):
       axes[i].imshow(slice.T, cmap="gray", origin="lower")
        
slice_0 = t2s_img_data[263, :, :]
slice_1 = t2s_img_data[:, 269, :]
slice_2 = t2s_img_data[:, :, 21]
show_slices([slice_0, slice_1, slice_2])
plt.suptitle("Center slices for T2* image")  

slice_0 = t2_img_data[32, :, :]
slice_1 = t2_img_data[:, 32, :]
slice_2 = t2_img_data[:, :, 7]
show_slices([slice_0, slice_1, slice_2])
plt.suptitle("Center slices for T2 image")  

In [ ]:
print(t2s_img.header)

In [ ]:
# Print the affine
print(t2s_img.affine)
print(t2_img.affine)

In [ ]:
t2_vox2t2s_vox = npl.inv(t2s_img.affine).dot(t2_img.affine)
t2_vox2t2s_vox.shape

#### Summarising Data

In [ ]:
root_dir = "Data/SCSeg/"
data_shapes = {}

print('Lesion Voxel Counts', '-'*20)
for root, dir, files in os.walk(root_dir):
    if len(files)>0:
        data_shapes[root] = {}
        print('Subject', root[11:13])
        for file in files:
            img = nib.load(root + '/' + file)
            img_data = img.get_fdata()
            data_shapes[root][file] = img_data.shape

            if file=='labelLesion.nii.gz':
                unique_vals, counts = np.unique(img_data.ravel(), return_counts=True)
                print(np.asarray((unique_vals, counts, np.round(100*counts/sum(counts), 6))).T)

In [ ]:
data_shapes

In [ ]:
for subj in data_shapes:
    if data_shapes[subj]['t2.nii.gz'] != (384, 384, 15):
        print(subj, data_shapes[subj]['t2.nii.gz'])
    

In [ ]:
plt.hist(img_data.ravel()) # (Just the last subject for now - T2*)